In [1]:
%run setting.ipynb

In [2]:
import lightgbm as lgb
import xgboost as xgb
import catboost as ctb

In [3]:
from sklearn import preprocessing
from sklearn.metrics import *

# training and test split date

In [4]:
test_start_date = '2023-06-01'

# load features

In [5]:
feature_name = np.load(
    os.path.join(
    data_path,
    'feature_name.npy',
    ),
)

categorical_feature = np.load(
    os.path.join(
    data_path,
    'categorical_feature.npy',
    ),
)

numemrical_feature = np.load(
    os.path.join(
    data_path,
    'numemrical_feature.npy',
    ),
)

# load data

In [6]:
data_filled = pd.read_parquet(
    os.path.join(
    data_path,
    'data_filled.parquet',
    )
)

data_encoded = pd.read_parquet(
    os.path.join(
    data_path,
    'data_encoded.parquet',
    )
)

In [7]:
X = data_filled[feature_name]

In [8]:
y = data_filled['label__rise']

In [9]:
Date = data_filled['Date']

In [10]:
data_encoded.drop(columns = ['level_0']).to_sql(
    "data_encoded", 
    conn,
    if_exists = "replace",
    )

9514

In [11]:
data_filled.drop(columns = ['level_0']).to_sql(
    "data_filled", 
    conn,
    if_exists = "replace",
    )

9514

# catboost model

## training test split

In [12]:
data_test = sql(f"""
select * 
from data_filled
where date(Date) >= '{test_start_date}'
""")

data_training = sql(f"""
select * 
from data_filled
where date(Date) < '{test_start_date}'
""")

X_training = data_training[feature_name]
y_training = data_training['label__rise']


X_test = data_test[feature_name]
y_test = data_test['label__rise']

In [19]:
print(f"""
trianing size:\t{X_training.shape}
test size:\t{X_test.shape}
""")


trianing size:	(9493, 72)
test size:	(21, 72)



## model training

In [13]:
%%time  

train_pool = ctb.Pool(
    X_training, 
    y_training, 
    cat_features = categorical_feature,
    )

params = {
    'loss_function': 'CrossEntropy',
    'task_type':"GPU",
    'devices':'0',
    }

ctb_model = ctb.CatBoostClassifier(**params)

ctb_model.fit(train_pool)

ctb_model.save_model(
    fname = os.path.join(
    data_path,
    "daily_rise.catboost"),
    )

0:	learn: 0.6927440	total: 54ms	remaining: 53.9s
1:	learn: 0.6922662	total: 105ms	remaining: 52.2s
2:	learn: 0.6916872	total: 154ms	remaining: 51s
3:	learn: 0.6913566	total: 202ms	remaining: 50.2s
4:	learn: 0.6907084	total: 250ms	remaining: 49.8s
5:	learn: 0.6903483	total: 298ms	remaining: 49.4s
6:	learn: 0.6899383	total: 346ms	remaining: 49.1s
7:	learn: 0.6894664	total: 395ms	remaining: 49s
8:	learn: 0.6891447	total: 442ms	remaining: 48.6s
9:	learn: 0.6887325	total: 488ms	remaining: 48.3s
10:	learn: 0.6882116	total: 536ms	remaining: 48.2s
11:	learn: 0.6879388	total: 585ms	remaining: 48.2s
12:	learn: 0.6875018	total: 633ms	remaining: 48.1s
13:	learn: 0.6870593	total: 681ms	remaining: 47.9s
14:	learn: 0.6868434	total: 726ms	remaining: 47.7s
15:	learn: 0.6865104	total: 773ms	remaining: 47.6s
16:	learn: 0.6862324	total: 821ms	remaining: 47.5s
17:	learn: 0.6859706	total: 872ms	remaining: 47.6s
18:	learn: 0.6856795	total: 918ms	remaining: 47.4s
19:	learn: 0.6854362	total: 962ms	remaining: 4

161:	learn: 0.6584575	total: 8.25s	remaining: 42.7s
162:	learn: 0.6583354	total: 8.3s	remaining: 42.6s
163:	learn: 0.6582559	total: 8.35s	remaining: 42.6s
164:	learn: 0.6581871	total: 8.42s	remaining: 42.6s
165:	learn: 0.6579375	total: 8.47s	remaining: 42.6s
166:	learn: 0.6578273	total: 8.53s	remaining: 42.5s
167:	learn: 0.6577609	total: 8.59s	remaining: 42.6s
168:	learn: 0.6575827	total: 8.64s	remaining: 42.5s
169:	learn: 0.6573240	total: 8.69s	remaining: 42.4s
170:	learn: 0.6571959	total: 8.74s	remaining: 42.4s
171:	learn: 0.6571282	total: 8.78s	remaining: 42.3s
172:	learn: 0.6570005	total: 8.83s	remaining: 42.2s
173:	learn: 0.6568392	total: 8.87s	remaining: 42.1s
174:	learn: 0.6567063	total: 8.92s	remaining: 42.1s
175:	learn: 0.6565629	total: 8.97s	remaining: 42s
176:	learn: 0.6563773	total: 9.04s	remaining: 42s
177:	learn: 0.6562551	total: 9.1s	remaining: 42s
178:	learn: 0.6561397	total: 9.14s	remaining: 41.9s
179:	learn: 0.6559143	total: 9.2s	remaining: 41.9s
180:	learn: 0.6558277

322:	learn: 0.6399978	total: 16.5s	remaining: 34.6s
323:	learn: 0.6398875	total: 16.6s	remaining: 34.5s
324:	learn: 0.6398398	total: 16.6s	remaining: 34.5s
325:	learn: 0.6397352	total: 16.7s	remaining: 34.4s
326:	learn: 0.6396313	total: 16.7s	remaining: 34.4s
327:	learn: 0.6395786	total: 16.8s	remaining: 34.4s
328:	learn: 0.6394028	total: 16.8s	remaining: 34.3s
329:	learn: 0.6392256	total: 16.9s	remaining: 34.3s
330:	learn: 0.6389430	total: 16.9s	remaining: 34.2s
331:	learn: 0.6387785	total: 17s	remaining: 34.2s
332:	learn: 0.6387017	total: 17s	remaining: 34.1s
333:	learn: 0.6385345	total: 17.1s	remaining: 34.1s
334:	learn: 0.6385315	total: 17.2s	remaining: 34.1s
335:	learn: 0.6384563	total: 17.2s	remaining: 34s
336:	learn: 0.6383423	total: 17.3s	remaining: 34s
337:	learn: 0.6381480	total: 17.3s	remaining: 33.9s
338:	learn: 0.6380517	total: 17.4s	remaining: 33.9s
339:	learn: 0.6379286	total: 17.4s	remaining: 33.8s
340:	learn: 0.6378228	total: 17.5s	remaining: 33.7s
341:	learn: 0.637714

481:	learn: 0.6252810	total: 24.1s	remaining: 25.9s
482:	learn: 0.6252021	total: 24.1s	remaining: 25.8s
483:	learn: 0.6251585	total: 24.2s	remaining: 25.8s
484:	learn: 0.6250662	total: 24.2s	remaining: 25.7s
485:	learn: 0.6249709	total: 24.3s	remaining: 25.7s
486:	learn: 0.6248569	total: 24.3s	remaining: 25.6s
487:	learn: 0.6246631	total: 24.4s	remaining: 25.6s
488:	learn: 0.6244693	total: 24.4s	remaining: 25.5s
489:	learn: 0.6242446	total: 24.5s	remaining: 25.5s
490:	learn: 0.6241366	total: 24.5s	remaining: 25.4s
491:	learn: 0.6240932	total: 24.6s	remaining: 25.4s
492:	learn: 0.6239694	total: 24.6s	remaining: 25.3s
493:	learn: 0.6239113	total: 24.7s	remaining: 25.3s
494:	learn: 0.6238036	total: 24.7s	remaining: 25.2s
495:	learn: 0.6238021	total: 24.8s	remaining: 25.2s
496:	learn: 0.6237321	total: 24.8s	remaining: 25.1s
497:	learn: 0.6237321	total: 24.9s	remaining: 25.1s
498:	learn: 0.6237117	total: 24.9s	remaining: 25s
499:	learn: 0.6235873	total: 24.9s	remaining: 24.9s
500:	learn: 0.

641:	learn: 0.6142624	total: 31.3s	remaining: 17.5s
642:	learn: 0.6141710	total: 31.3s	remaining: 17.4s
643:	learn: 0.6140723	total: 31.4s	remaining: 17.4s
644:	learn: 0.6139216	total: 31.5s	remaining: 17.3s
645:	learn: 0.6139215	total: 31.5s	remaining: 17.3s
646:	learn: 0.6139200	total: 31.5s	remaining: 17.2s
647:	learn: 0.6139180	total: 31.6s	remaining: 17.2s
648:	learn: 0.6137382	total: 31.6s	remaining: 17.1s
649:	learn: 0.6137373	total: 31.7s	remaining: 17s
650:	learn: 0.6137353	total: 31.7s	remaining: 17s
651:	learn: 0.6136669	total: 31.8s	remaining: 17s
652:	learn: 0.6136640	total: 31.8s	remaining: 16.9s
653:	learn: 0.6136387	total: 31.9s	remaining: 16.9s
654:	learn: 0.6136379	total: 31.9s	remaining: 16.8s
655:	learn: 0.6135293	total: 32s	remaining: 16.8s
656:	learn: 0.6135294	total: 32s	remaining: 16.7s
657:	learn: 0.6134055	total: 32s	remaining: 16.7s
658:	learn: 0.6134055	total: 32.1s	remaining: 16.6s
659:	learn: 0.6134054	total: 32.1s	remaining: 16.5s
660:	learn: 0.6134033	to

804:	learn: 0.6058054	total: 38.7s	remaining: 9.38s
805:	learn: 0.6058050	total: 38.7s	remaining: 9.32s
806:	learn: 0.6058046	total: 38.8s	remaining: 9.27s
807:	learn: 0.6058043	total: 38.8s	remaining: 9.22s
808:	learn: 0.6056402	total: 38.9s	remaining: 9.18s
809:	learn: 0.6056389	total: 38.9s	remaining: 9.13s
810:	learn: 0.6056330	total: 39s	remaining: 9.08s
811:	learn: 0.6055203	total: 39s	remaining: 9.03s
812:	learn: 0.6055203	total: 39s	remaining: 8.98s
813:	learn: 0.6055165	total: 39.1s	remaining: 8.93s
814:	learn: 0.6054952	total: 39.1s	remaining: 8.88s
815:	learn: 0.6054949	total: 39.2s	remaining: 8.84s
816:	learn: 0.6054930	total: 39.2s	remaining: 8.79s
817:	learn: 0.6054926	total: 39.3s	remaining: 8.74s
818:	learn: 0.6054911	total: 39.3s	remaining: 8.69s
819:	learn: 0.6054908	total: 39.4s	remaining: 8.64s
820:	learn: 0.6054908	total: 39.4s	remaining: 8.59s
821:	learn: 0.6054898	total: 39.4s	remaining: 8.54s
822:	learn: 0.6053336	total: 39.5s	remaining: 8.49s
823:	learn: 0.6053

966:	learn: 0.5979038	total: 46.1s	remaining: 1.57s
967:	learn: 0.5979007	total: 46.2s	remaining: 1.53s
968:	learn: 0.5978986	total: 46.2s	remaining: 1.48s
969:	learn: 0.5978972	total: 46.3s	remaining: 1.43s
970:	learn: 0.5978972	total: 46.3s	remaining: 1.38s
971:	learn: 0.5977638	total: 46.4s	remaining: 1.34s
972:	learn: 0.5976538	total: 46.5s	remaining: 1.29s
973:	learn: 0.5976534	total: 46.5s	remaining: 1.24s
974:	learn: 0.5975463	total: 46.5s	remaining: 1.19s
975:	learn: 0.5974534	total: 46.6s	remaining: 1.15s
976:	learn: 0.5974448	total: 46.7s	remaining: 1.1s
977:	learn: 0.5973197	total: 46.7s	remaining: 1.05s
978:	learn: 0.5971273	total: 46.8s	remaining: 1s
979:	learn: 0.5971271	total: 46.8s	remaining: 955ms
980:	learn: 0.5969980	total: 46.8s	remaining: 907ms
981:	learn: 0.5969191	total: 46.9s	remaining: 860ms
982:	learn: 0.5967213	total: 47s	remaining: 812ms
983:	learn: 0.5967208	total: 47s	remaining: 764ms
984:	learn: 0.5964721	total: 47s	remaining: 716ms
985:	learn: 0.5963511	

## evaluation

In [48]:
y_test_pred = ctb_model.predict_proba(X_test)[:,1]
y_test_pred_label = ctb_model.predict(X_test)

fpr, tpr, thresholds = roc_curve(
    y_test,
    y_test_pred, 
    pos_label=1)
test_auc_ctb = auc(fpr, tpr)

test_average_precision_ctb = average_precision_score(
    y_test,
    y_test_pred, 
    )

test_acc_ctb = average_precision_score(
    y_test,
    y_test_pred_label, 
    )

test_f1_ctb = f1_score(
    y_test,
    y_test_pred_label, 
    )

test_recall_ctb = recall_score(
    y_test,
    y_test_pred_label, 
    )

test_precision_ctb = precision_score(
    y_test,
    y_test_pred_label, 
    )

In [49]:
y_training_pred = ctb_model.predict_proba(X_training)[:,1]
y_training_pred_label = ctb_model.predict(X_training)

fpr, tpr, thresholds = roc_curve(
    y_training,
    y_training_pred, 
    pos_label=1)
training_auc_ctb = auc(fpr, tpr)

training_average_precision_ctb = average_precision_score(
    y_training,
    y_training_pred, 
    )

training_acc_ctb = average_precision_score(
    y_training,
    y_training_pred_label, 
    )

training_f1_ctb = f1_score(
    y_training,
    y_training_pred_label, 
    )

training_recall_ctb = recall_score(
    y_training,
    y_training_pred_label, 
    )

training_precision_ctb = precision_score(
    y_training,
    y_training_pred_label, 
    )

In [51]:
metric_df = [
    {
    'metric':'AUC',
    'training':training_auc_ctb,
    'test':test_auc_ctb
    },
    {
    'metric':'Average Precision',
    'training':training_average_precision_ctb,
    'test':test_average_precision_ctb
    },  
    {
    'metric':'Accuracy',
    'training':training_acc_ctb,
    'test':test_acc_ctb
    }, 
    {
    'metric':'F1',
    'training':training_f1_ctb,
    'test':test_f1_ctb
    },
    {
    'metric':'Recall',
    'training':training_recall_ctb,
    'test':test_recall_ctb
    },
    {
    'metric':'Precision',
    'training':training_precision_ctb,
    'test':test_precision_ctb
    },
]

pd.DataFrame(metric_df)

,metric,training,test
0,AUC,0.858549,0.408163
1,Average Precision,0.874874,0.641751
2,Accuracy,0.681195,0.638095
3,F1,0.796971,0.500000
4,Recall,0.944044,0.428571
5,Precision,0.689547,0.600000


In [52]:
test_positive_rate = np.sum(y_test)/len(y_test)
print(f"test_positive_rate:{test_positive_rate:0.4f}")

test_positive_rate:0.6667


## feature importance

In [16]:
feature_importance = []

for n, i in zip(ctb_model.feature_names_, ctb_model.feature_importances_):
    feature_importance.append({
        'feature_name':n,
        'feature_importance':i
    })
    
pd.DataFrame(feature_importance).to_sql(
    "feature_importance_catboost_rise", 
    conn,
    if_exists = "replace",
    )

72

In [17]:
sql(f"""
with total_weight as (
select sum(feature_importance) as feature_importance
from feature_importance_catboost_rise
)

select a.*,
a.feature_importance/b.feature_importance as feature_importance_percentage
from feature_importance_catboost_rise as a,
total_weight as b
--where a.feature_name like '%rise%'
order by a.feature_importance desc
""")

,index,feature_name,feature_importance,feature_importance_percentage
0,13,feature_num__close_to_open_ratio__8_day_ago,6.448014,0.064480
1,46,feature_num__high_to_low_ratio__4_day_ago,5.721143,0.057211
2,69,feature_num__close_to_open_ratio__1_day_ago,4.890747,0.048907
3,61,feature_num__close_to_open_ratio__2_day_ago,4.316801,0.043168
4,21,feature_num__close_to_open_ratio__7_day_ago,4.248552,0.042486
...,...,...,...,...
67,17,feature_num__Close__7_day_ago,0.132500,0.001325
68,23,feature_cat__daily_rise__7_day_ago,0.119941,0.001199
69,34,feature_num__High__5_day_ago,0.111750,0.001118
70,42,feature_num__High__4_day_ago,0.063162,0.000632


# end